This one: https://colab.research.google.com/drive/1qPDsg42_trAny4ys3qoh6veoRogL5U-O?usp=sharing

Full one: https://colab.research.google.com/drive/11i08nuDCRhlHF0NKjtO6cfDjtOh0Va7I?usp=sharing

**Paper from which the idea came from:** https://arxiv.org/pdf/2002.10558.pdf

**Implementation from the paper (Tensorflow 1):** https://github.com/Raocp/PINN-laminar-flow

## Neural Network Classes

In [ ]:
%pip install pyDOE

In [ ]:
%pip install tensorflow-addons

In [4]:
import os
import pickle
import random
import time

import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from pyDOE import lhs
import ipywidgets as widgets

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.FATAL)

In [ ]:
if tf.test.is_gpu_available():
    print("GPU available")
    print("GPU:", tf.config.list_physical_devices("GPU"))
else:
    print("GPU is not avaulable")

### NN

In [5]:
class PINN(Model):
    def __init__(self, output_dim, num_hidden_layers, num_neurons_per_layer, activation, kernel_initializer, dropout_rate, l2_lambda, **kwargs):
        super().__init__(**kwargs)
        self.num_hidden_layers = num_hidden_layers
        self.output_dim = output_dim
        self.hidden = [
            [
                Dense(
                    num_neurons_per_layer,
                    activation=tf.keras.activations.get(activation),
                    kernel_initializer=kernel_initializer,
                    # this just didn't help at all
                    #kernel_regularizer=regularizers.l2(l2_lambda)
                ),
                #BatchNormalization(),
                #Dropout(dropout_rate)
            ]
            for _ in range(self.num_hidden_layers)
        ]
        self.out = Dense(output_dim)

    def call(self, inputs):
        x = inputs
        for i in range(self.num_hidden_layers):
            x = self.hidden[i][0](x)
            #x = self.hidden[i][1](x)
            #x = self.hidden[i][2](x)
        return self.out(x)


### PINN Solver for PDE

In [6]:
def PicsInProcess(xmin, xmax, ymin, ymax, field_MIXED, s=2, alpha=0.5, marker='o', filename=f"uvp.png"):

    [x_MIXED, y_MIXED, u_MIXED, v_MIXED, p_MIXED] = field_MIXED

    fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(7, 4))
    fig.subplots_adjust(hspace=0.2, wspace=0.2)
    def add_step_lines(ax):
            ax.plot([1.0, 1.5], [0.21, 0.21], color='black', linewidth=2) # lower horizontal line
            ax.plot([0.55, 1.0], [0, 0.21], color='black', linewidth=2) # vertical line

    # Define a function to filter points outside the channel
    def filter_points(x, y, u, v, p):
        mask = np.logical_or((x < 1.0) & (y <= 0.41), (x >= 1.0) & (y > 0.21))
        return x[mask], y[mask], u[mask], v[mask], p[mask]

    # Filter points outside the channel
    x_MIXED, y_MIXED, u_MIXED, v_MIXED, p_MIXED = filter_points(x_MIXED, y_MIXED, u_MIXED, v_MIXED, p_MIXED)


    # Plot MIXED result
    cf = ax[0, 0].scatter(x_MIXED, y_MIXED, c=u_MIXED, alpha=alpha-0.1, edgecolors='none', cmap='rainbow', marker=marker, s=int(s))
    ax[0, 0].axis('square')
    for key, spine in ax[0, 0].spines.items():
        if key in ['right','top','left','bottom']:
            spine.set_visible(False)
    ax[0, 0].set_xticks([])
    ax[0, 0].set_yticks([])
    ax[0, 0].set_xlim([xmin, xmax])
    ax[0, 0].set_ylim([ymin, ymax])

    ax[0, 0].set_title(r'$u$ (m/s)')
    fig.colorbar(cf, ax=ax[0, 0], fraction=0.046, pad=0.04)

    cf = ax[1, 0].scatter(x_MIXED, y_MIXED, c=v_MIXED, alpha=alpha-0.1, edgecolors='none', cmap='rainbow', marker=marker, s=int(s))
    ax[1, 0].axis('square')
    for key, spine in ax[1, 0].spines.items():
        if key in ['right','top','left','bottom']:
            spine.set_visible(False)
    ax[1, 0].set_xticks([])
    ax[1, 0].set_yticks([])
    ax[1, 0].set_xlim([xmin, xmax])
    ax[1, 0].set_ylim([ymin, ymax])

    ax[1, 0].set_title(r'$v$ (m/s)')
    fig.colorbar(cf, ax=ax[1, 0], fraction=0.046, pad=0.04)

    cf = ax[2, 0].scatter(x_MIXED, y_MIXED, c=p_MIXED, alpha=alpha, edgecolors='none', cmap='rainbow', marker=marker, s=int(s), vmin=-0.25, vmax=4.0)
    ax[2, 0].axis('square')
    for key, spine in ax[2, 0].spines.items():
        if key in ['right','top','left','bottom']:
            spine.set_visible(False)
    ax[2, 0].set_xticks([])
    ax[2, 0].set_yticks([])
    ax[2, 0].set_xlim([xmin, xmax])
    ax[2, 0].set_ylim([ymin, ymax])

    ax[2, 0].set_title('Pressure (Pa)')
    fig.colorbar(cf, ax=ax[2, 0], fraction=0.046, pad=0.04)
    plt.show()
    plt.close('all')

In [7]:
def plot_Wall_Shear_stress(solver):

    import numpy as np
    import matplotlib.pyplot as plt

    # counting wall shear stress
    wall_shear_stress, x_WALL = solver.compute_wall_shear_stress()

    x_WALL = x_WALL.numpy()
    wall_shear_stress = wall_shear_stress.numpy()
    sort_indices = np.argsort(x_WALL.flatten())
    x_WALL = x_WALL.flatten()[sort_indices]
    wall_shear_stress= wall_shear_stress.flatten()[sort_indices]

    plt.figure(figsize=(10, 5))
    plt.plot(x_WALL, wall_shear_stress, label='Wall Shear Stress')
    plt.xlabel('x')
    plt.ylabel('Shear stress')
    plt.legend()

    # saving the plot
    plt.savefig('wall_shear_stress.png')

    # saving it in txt
    np.savetxt('wall_shear_stress.txt', np.column_stack((x_WALL, wall_shear_stress)))

In [8]:
#saves model in zip, plots wall shear stress, also loss history in pickle
def saveNN(solver):
    solver.model.save("saved_model")
    !zip -r /content/saved_model.zip /content/saved_model
    losses_per_iteration = solver.get_losses_per_iteration()
    with open(f'losses_history.pickle', 'wb') as f:
        pickle.dump(losses_per_iteration, f)
    plot_Wall_Shear_stress(solver)

In [9]:
class PINNSolver():

    def __init__(self, model, Collo, INLET, OUTLET, WALL, mu=0.02, beta=2, data = None, alpha = 1):

        self.losses_per_iteration = []
        self.model = model
        self.beta = beta
        #coef for data
        self.alpha = alpha
        # Store math constants
        self.rho = 1.0
        self.mu = mu

        # Store collocation points
        self.x_c = Collo[:, 0:1]
        self.y_c = Collo[:, 1:2]

        # if data was provided for solver we initialise the following variables
        if data is not None:
            self.x_data = data["    x-coordinate"].values
            self.y_data = data["    y-coordinate"].values
            self.x_data = tf.convert_to_tensor(self.x_data, dtype=tf.float32)
            self.y_data = tf.convert_to_tensor(self.y_data, dtype=tf.float32)
            self.x_data = tf.reshape(self.x_data, (-1, 1))
            self.y_data = tf.reshape(self.y_data, (-1, 1))
            self.u_data = data["      x-velocity"].values
            self.v_data = data["      y-velocity"].values
            self.p_data = data["        pressure"].values

        # Store inlet points
        self.x_INLET = INLET[:, 0:1]
        self.y_INLET = INLET[:, 1:2]
        self.u_INLET = INLET[:, 2:3]
        self.v_INLET = INLET[:, 3:4]

        # Store outlet points
        self.x_OUTLET = OUTLET[:, 0:1]
        self.y_OUTLET = OUTLET[:, 1:2]

        # Store wall points
        self.x_WALL = WALL[:, 0:1]
        self.y_WALL = WALL[:, 1:2]

        # some variables to plot fields during while the model learns
        x_PINN_1 = np.linspace(0, 1.0, 221)
        y_PINN_1 = np.linspace(0, 0.21, 101)
        x_PINN_2 = np.linspace(1.0, 1.5, 241)
        y_PINN_2 = np.linspace(0.21, 0.41, 101)
        x_PINN = np.concatenate((x_PINN_1, x_PINN_2))
        y_PINN = np.concatenate((y_PINN_1, y_PINN_2))
        x_PINN, y_PINN = np.meshgrid(x_PINN, y_PINN)
        self.x_PINN = x_PINN.flatten()[:, None]
        self.y_PINN = y_PINN.flatten()[:, None]

    def compute_total_loss(self):
        beta = self.beta

        fn_loss = self.get_loss_part("fn")
        wall_loss = self.get_loss_part("wall")
        inlet_loss = self.get_loss_part("inlet")
        outlet_loss = self.get_loss_part("outlet")
        data_loss = self.get_loss_part("data")
        total_loss = fn_loss + beta * (wall_loss + inlet_loss + outlet_loss) + self.alpha * data_loss
        return total_loss


    def get_output(self, x, y):
        with tf.GradientTape(persistent=True) as tape:
            tape.watch([x, y])

            psips = self.model(tf.concat([x, y], axis=1))
            psi, p, s11, s22, s12 = tf.split(psips, [1, 1, 1, 1, 1], axis=1)
            u = tape.gradient(psi, y)
            v = -tape.gradient(psi, x)

        return psi, p, s11, s22, s12, tape, x, y, u, v


    def get_loss_part(self, part):
        if part == "wall":
            _, _, _, _, _, tape, _, _, u, v = self.get_output(x=self.x_WALL, y=self.y_WALL)
            loss = tf.reduce_mean(tf.square(u)) + tf.reduce_mean(tf.square(v))
        elif part == "inlet":
            _, _, _, _, _, tape, _, _, u, v = self.get_output(x=self.x_INLET, y=self.y_INLET)
            loss = tf.reduce_mean(tf.square(u - self.u_INLET)) + tf.reduce_mean(tf.square(v - self.v_INLET))
        elif part == "outlet":
            _, p, _, _, _, tape, _, _, _, _ = self.get_output(x=self.x_OUTLET, y=self.y_OUTLET)
            loss = tf.reduce_mean(tf.square(p))
        if part == "data":
            data_loss = 0
            if hasattr(self, 'x_data') and hasattr(self, 'y_data'):
                _, p, _, _, _, _, _, _, u, v = self.get_output(self.x_data, self.y_data)
                data_loss = tf.reduce_mean(tf.square(u - self.u_data)) + tf.reduce_mean(tf.square(v - self.v_data)) + tf.reduce_mean(tf.square(p - self.p_data))
            return data_loss
        if part == "fn":
            psi, p, s11, s22, s12, tape, x, y, u, v = self.get_output(x=self.x_c, y=self.y_c)

            u_x, u_y = tape.gradient(u, [x, y])
            v_x, v_y = tape.gradient(v, [x, y])

            s11_x = tape.gradient(s11, x)
            s12_x, s12_y = tape.gradient(s12, [x, y])
            s22_y = tape.gradient(s22, y)

            # Sigma residuals
            r_s11 = -p + 2 * self.mu * u_x - s11
            r_s22 = -p + 2 * self.mu * v_y - s22
            r_s12 = self.mu * (u_y + v_x) - s12

            # Pressure residual
            r_p = p + (s11 + s22) / 2

            # navier stokes equation residuals
            r_u = self.rho * (u * u_x + v * u_y) - s11_x - s12_y
            r_v = self.rho * (u * v_x + v * v_y) - s12_x - s22_y

            loss = (
                tf.reduce_mean(tf.square(r_s11)) +
                tf.reduce_mean(tf.square(r_s22)) +
                tf.reduce_mean(tf.square(r_s12)) +
                tf.reduce_mean(tf.square(r_p)) +
                tf.reduce_mean(tf.square(r_u)) +
                tf.reduce_mean(tf.square(r_v))
            )
        del tape
        return loss


    def solve_with_TFoptimizer(self, optimizer, N):
        with tf.device("/GPU:0"):

            for iteration in range(N):
                with tf.GradientTape(persistent=True) as tape:
                    tape.watch(self.model.trainable_variables)
                    loss = self.compute_total_loss()
                self.losses_per_iteration.append(loss.numpy())
                grads = tape.gradient(loss, self.model.trainable_variables)
                del tape

                optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

                print(f"Iteration: {iteration} || Loss: {loss:.10f}")
                if iteration%1000 == 0:
                    psi, p_PINN, s11, s22, s12, tape, x, y, u_PINN, v_PINN = self.get_output(tf.convert_to_tensor(self.x_PINN), tf.convert_to_tensor(self.y_PINN))
                    field_MIXED = [self.x_PINN, self.y_PINN, u_PINN, v_PINN, p_PINN]
                    PicsInProcess(xmin=0, xmax=1.5, ymin=0, ymax=0.41, field_MIXED=field_MIXED, s=2, alpha=0.5, marker='o', filename=f"uvp.png")
                if iteration%10_000 == 0 and iteration != 0:
                    saveNN(self)

    def compute_total_loss_and_gradients(self, X):
        with tf.GradientTape() as tape:
            loss = self.compute_total_loss()
        grads = tape.gradient(loss, self.model.trainable_variables)
        return loss, grads

    def get_losses_per_iteration(self):
        return self.losses_per_iteration

    def get_loss_to_data(self, pd_data=None):
        data_loss = None
        if pd_data is not None:
            x_data = pd_data["    x-coordinate"].values.astype(np.float32)
            y_data = pd_data["    y-coordinate"].values.astype(np.float32)
            x_data = tf.convert_to_tensor(x_data, dtype=tf.float32)
            y_data = tf.convert_to_tensor(y_data, dtype=tf.float32)
            x_data = tf.reshape(x_data, (-1, 1))
            y_data = tf.reshape(y_data, (-1, 1))
            u_data = pd_data["      x-velocity"].values.astype(np.float32)
            v_data = pd_data["      y-velocity"].values.astype(np.float32)
            p_data = pd_data["        pressure"].values.astype(np.float32)
            _, p, _, _, _, _, _, _, u, v = self.get_output(x_data, y_data)

            u_mae_percent = tf.reduce_mean(tf.abs((u - u_data))) / tf.reduce_max(tf.abs(u_data)) * 100
            v_mae_percent = tf.reduce_mean(tf.abs((v - v_data))) / tf.reduce_max(tf.abs(v_data)) * 100
            p_mae_percent = tf.reduce_mean(tf.abs((p - p_data))) / tf.reduce_max(tf.abs(p_data)) * 100

            print(f"u_mae_perc: {u_mae_percent}, v_mae_perc: {v_mae_percent}, p_mae_perc: {p_mae_percent}")
            data_loss = u_mae_percent + v_mae_percent + p_mae_percent
            return data_loss
        return data_loss


    def compute_wall_shear_stress(self):
        mask = tf.logical_and(self.y_WALL == 0.00, tf.logical_and(self.x_WALL >= 0, self.x_WALL <= 1))
        x_WALL = tf.boolean_mask(self.x_WALL, mask)
        y_WALL = tf.boolean_mask(self.y_WALL, mask)

        x_WALL = tf.reshape(x_WALL, [-1, 1])
        y_WALL = tf.reshape(y_WALL, [-1, 1])

        with tf.GradientTape(persistent=True) as tape:
            tape.watch([x_WALL, y_WALL])
            _, _, _, _, _, _, _, _, u, _ = self.get_output(x_WALL, y_WALL)
            du_dy = tape.gradient(u, y_WALL)
        wall_shear_stress = self.mu * du_dy
        return wall_shear_stress, x_WALL


## Setup

### Network Setup

### New Model

In [10]:
output_dim = 5
num_hidden_layers = 8
num_neurons_per_layer = 40
activation = 'tanh'
kernel_initializer = 'glorot_uniform'

In [11]:
model = PINN(output_dim, num_hidden_layers, num_neurons_per_layer, activation, kernel_initializer, dropout_rate=0.5, l2_lambda=0.1)
model.build(input_shape=(None, 2))

### Loading Model

In [ ]:
#!gdown 1QkKu7t_9Mad52eGIJu88pPsdcNKVdGe- #Re=14_Iters=260_000_hid=8_neurons=40
#!gdown 1rB2PTazlkO70Lei5WZ3YYZ2z33jfs09b #Re=100_iters=90_000_hid=8_neurons=40

In [ ]:
!unzip saved_model.zip

In [ ]:
model = tf.keras.models.load_model("/content/saved_model")

### Solver Setup

In [12]:
# Domain bounds
lb = np.array([0, 0])
ub = np.array([1.5, 0.41])

#WALLS
wall_step_up = [1.0, 0.0] + [0.0, 0.21] * lhs(2, 201).astype(np.float32)
wall_step_right = [1.0, 0.21] + [0.5, 0.0] * lhs(2, 441).astype(np.float32)
wall_up = [0.0, 0.41] + [1.5, 0.0] * lhs(2, 441).astype(np.float32)
wall_lw = [0.0, 0.00] + [1.0, 0.0] * lhs(2, 441).astype(np.float32)
WALL = tf.concat((wall_step_up, wall_step_right, wall_up, wall_lw), axis=0)
WALL = tf.cast(WALL, dtype=tf.float32)

# INLET
INLET = ([0.0, 0.0] + [0.0, 0.41] * lhs(2, 201)).astype(np.float32)
U_max = 1.0
y_INLET = INLET[:,1:2]
u_INLET = 4 * U_max * y_INLET * (0.41 - y_INLET) / (0.41 ** 2)
v_INLET = 0 * y_INLET
INLET = tf.concat((INLET, u_INLET, v_INLET), axis=1)

# OUTLET
OUTLET = tf.convert_to_tensor([1.5, 0.21] + [0.0, 0.20] * lhs(2, 201), dtype=tf.float32)

In [13]:
# Collocation points for equation residual
XY_c_before = [0.0, 0.0] + [1.0, 0.41] * lhs(2, 40000).astype(np.float32)
XY_c_after = [1.0, 0.21] + [0.5, 0.2] * lhs(2, 10000).astype(np.float32)
XY_c = np.concatenate((XY_c_before, XY_c_after), 0)
XY_c = tf.concat((XY_c, WALL, OUTLET, INLET[:,0:2]), 0)

Collo = XY_c

In [ ]:
# Visualize the collocation points
fig, ax = plt.subplots()
ax.set_aspect('equal')
plt.scatter(XY_c[:,0:1], XY_c[:,1:2], marker='o', alpha=0.1 ,color='blue')
plt.scatter(WALL[:,0:1], WALL[:,1:2], marker='o', alpha=0.2 , color='green')
plt.scatter(OUTLET[:, 0:1], OUTLET[:, 1:2], marker='o', alpha=0.2, color='orange')
plt.scatter(INLET[:, 0:1], INLET[:, 1:2], marker='o', alpha=0.2, color='red')
plt.show()

In [ ]:
#!gdown 1S9UMttJSLnCt9qMxZwqPI0p8ta3V1JO- # Re = 10 (mu = 0.02738)
!gdown 1vY6l22eNzqvJH7CPPCBEsAC4ddoqHJoU # Re = 14 (mu = 0.02)
#!gdown 1IPJkcPMiluQt2B4YqCCnsOl4l6KSlEAu # Re = 50 (mu = 0.005476)
#!gdown 1AUkiHKL1b69qCs9UFRg1IAbXvDGQj5_v # Re = 75 (mu = 0.0036507)
#!gdown 1DQuGXahCaQ75NBTHzEkJZshGXOFIky4Y # Re = 100 (mu = 0.002738)
#!gdown 1zB0R5C0QCSX-05p2mnrzGdPdsVdkDSqr # Re = 250 (mu = 0.0010952)
#!gdown 1lJQBfwJcR_mwYx1JDKBhPKSQ6ZCsMA54 # Re = 500 (mu = 0.00054761)

In [16]:
#import pandas as pd
#data = pd.read_csv('FluentSolStep.txt')
data=None

In [17]:
solver = PINNSolver(model, Collo, INLET, OUTLET,  WALL, mu = 0.02, beta=2, data=data, alpha=1)
# last_loss = solver.compute_total_loss()
# print(f"Last Loss: {last_loss:.10f}")
# df = pd.read_csv('FluentSolStep.txt')
# lost_to_data = solver.get_loss_to_data(df)

## Learning Process

In [ ]:
from tensorflow_addons.optimizers import CyclicalLearningRate

# Definening min and max learning rate
INIT_LR = 1e-5
MAX_LR = 1e-2

# scaling function for learning rate
scale_fn = lambda x: tf.math.exp(-x)
#Other Variants:
#1
#tf.math.exp(-x)
#1/(2.**(x-1))

#creating clr
clr = CyclicalLearningRate(initial_learning_rate=INIT_LR,
                           maximal_learning_rate=MAX_LR,
                           step_size=2000,
                           scale_fn=scale_fn,
                           scale_mode='cycle')

In [19]:
optimizer = tf.keras.optimizers.Adamax(learning_rate = clr, beta_1 = 0.9, beta_2=0.999)
optim = optimizer

In [ ]:
solver.solve_with_TFoptimizer(optim, N=1_001)

## Results Output

In [ ]:
#plot loss per iteration
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
plt.plot(solver.losses_per_iteration)
plt.title('Loss per iteration')
plt.yscale('log')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.grid(True)
plt.show()

In [ ]:
#!gdown 1S9UMttJSLnCt9qMxZwqPI0p8ta3V1JO- # Re = 10 (mu = 0.02738)
!gdown 1vY6l22eNzqvJH7CPPCBEsAC4ddoqHJoU # Re = 14 (mu = 0.02)
#!gdown 1IPJkcPMiluQt2B4YqCCnsOl4l6KSlEAu # Re = 50 (mu = 0.005476)
#!gdown 1AUkiHKL1b69qCs9UFRg1IAbXvDGQj5_v # Re = 75 (mu = 0.0036507)
#!gdown 1DQuGXahCaQ75NBTHzEkJZshGXOFIky4Y # Re = 100 (mu = 0.002738)
#!gdown 1zB0R5C0QCSX-05p2mnrzGdPdsVdkDSqr # Re = 250 (mu = 0.0010952)
#!gdown 1lJQBfwJcR_mwYx1JDKBhPKSQ6ZCsMA54 # Re = 500 (mu = 0.00054761)

In [ ]:
#last loss value
last_loss = solver.compute_total_loss()
print(f"Last Loss: {last_loss:.10f}")

In [ ]:
#loss to data value (percent summ)
import pandas as pd
df = pd.read_csv('FluentSolStep.txt')
loss_to_data = solver.get_loss_to_data(df)
print(loss_to_data)

In [ ]:
#functions to plot the fields

def postProcess(xmin, xmax, ymin, ymax, field_FLUENT, field_MIXED, s=2, alpha=0.5, marker='o', filename=f"uvp.png"):

    [x_FLUENT, y_FLUENT, u_FLUENT, v_FLUENT, p_FLUENT] = field_FLUENT
    [x_MIXED, y_MIXED, u_MIXED, v_MIXED, p_MIXED] = field_MIXED

    fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(7, 4))
    fig.subplots_adjust(hspace=0.2, wspace=0.2)
    def add_step_lines(ax):
            ax.plot([1.0, 1.5], [0.21, 0.21], color='black', linewidth=2) # lower horizontal line
            ax.plot([0.55, 1.0], [0, 0.21], color='black', linewidth=2) # vertical line

    # Define a function to filter points outside the channel
    def filter_points(x, y, u, v, p):
        mask = np.logical_or((x < 1.0) & (y <= 0.41), (x >= 1.0) & (y > 0.21))
        return x[mask], y[mask], u[mask], v[mask], p[mask]

    # Filter points outside the channel
    x_MIXED, y_MIXED, u_MIXED, v_MIXED, p_MIXED = filter_points(x_MIXED, y_MIXED, u_MIXED, v_MIXED, p_MIXED)

    #Plot PINN result
    cf = ax[0, 0].scatter(x_MIXED, y_MIXED, c=u_MIXED, alpha=alpha-0.1, edgecolors='none', cmap='rainbow', marker=marker, s=int(s))
    ax[0, 0].axis('square')
    for key, spine in ax[0, 0].spines.items():
        if key in ['right','top','left','bottom']:
            spine.set_visible(False)
    ax[0, 0].set_xticks([])
    ax[0, 0].set_yticks([])
    ax[0, 0].set_xlim([xmin, xmax])
    ax[0, 0].set_ylim([ymin, ymax])
    ax[0, 0].set_title(r'$u$ (m/s)')
    fig.colorbar(cf, ax=ax[0, 0], fraction=0.046, pad=0.04)

    cf = ax[1, 0].scatter(x_MIXED, y_MIXED, c=v_MIXED, alpha=alpha-0.1, edgecolors='none', cmap='rainbow', marker=marker, s=int(s))
    ax[1, 0].axis('square')
    for key, spine in ax[1, 0].spines.items():
        if key in ['right','top','left','bottom']:
            spine.set_visible(False)
    ax[1, 0].set_xticks([])
    ax[1, 0].set_yticks([])
    ax[1, 0].set_xlim([xmin, xmax])
    ax[1, 0].set_ylim([ymin, ymax])
    ax[1, 0].set_title(r'$v$ (m/s)')
    fig.colorbar(cf, ax=ax[1, 0], fraction=0.046, pad=0.04)

    cf = ax[2, 0].scatter(x_MIXED, y_MIXED, c=p_MIXED, alpha=alpha, edgecolors='none', cmap='rainbow', marker=marker, s=int(s), vmin=-0.25, vmax=4.0)
    ax[2, 0].axis('square')
    for key, spine in ax[2, 0].spines.items():
        if key in ['right','top','left','bottom']:
            spine.set_visible(False)
    ax[2, 0].set_xticks([])
    ax[2, 0].set_yticks([])
    ax[2, 0].set_xlim([xmin, xmax])
    ax[2, 0].set_ylim([ymin, ymax])
    ax[2, 0].set_title('Pressure (Pa)')
    fig.colorbar(cf, ax=ax[2, 0], fraction=0.046, pad=0.04)


    # Plot FLUENT result
    cf = ax[0, 1].scatter(x_FLUENT, y_FLUENT, c=u_FLUENT, alpha=alpha, edgecolors='none', cmap='rainbow', marker=marker, s=s)
    ax[0, 1].axis('square')
    for key, spine in ax[0, 1].spines.items():
        if key in ['right','top','left','bottom']:
            spine.set_visible(False)
    ax[0, 1].set_xticks([])
    ax[0, 1].set_yticks([])
    ax[0, 1].set_xlim([xmin, xmax])
    ax[0, 1].set_ylim([ymin, ymax])
    ax[0, 1].set_title(r'$u$ (m/s)')
    fig.colorbar(cf, ax=ax[0, 1], fraction=0.046, pad=0.04)

    cf = ax[1, 1].scatter(x_FLUENT, y_FLUENT, c=v_FLUENT, alpha=alpha, edgecolors='none', cmap='rainbow', marker=marker, s=s)
    ax[1, 1].axis('square')
    for key, spine in ax[1, 1].spines.items():
        if key in ['right','top','left','bottom']:
            spine.set_visible(False)
    ax[1, 1].set_xticks([])
    ax[1, 1].set_yticks([])
    ax[1, 1].set_xlim([xmin, xmax])
    ax[1, 1].set_ylim([ymin, ymax])
    ax[1, 1].set_title(r'$v$ (m/s)')
    fig.colorbar(cf, ax=ax[1, 1], fraction=0.046, pad=0.04)

    cf = ax[2, 1].scatter(x_FLUENT, y_FLUENT, c=p_FLUENT, alpha=alpha, edgecolors='none', cmap='rainbow', marker=marker, s=s, vmin=-0.25, vmax=4.0)
    ax[2, 1].axis('square')
    for key, spine in ax[2, 1].spines.items():
        if key in ['right','top','left','bottom']:
            spine.set_visible(False)
    ax[2, 1].set_xticks([])
    ax[2, 1].set_yticks([])
    ax[2, 1].set_xlim([xmin, xmax])
    ax[2, 1].set_ylim([ymin, ymax])
    ax[2, 1].set_title('Pressure (Pa)')
    fig.colorbar(cf, ax=ax[2, 1], fraction=0.046, pad=0.04)

    plt.savefig(f'./{filename}', dpi=300)
    plt.show()
    plt.close('all')

def preprocess(dir='FluentSolStep.txt'):
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.interpolate import griddata

    df = pd.read_csv('FluentSolStep.txt')
    x = df["    x-coordinate"].values
    y = df["    y-coordinate"].values
    u = df["      x-velocity"].values
    v = df["      y-velocity"].values
    p = df["        pressure"].values

    # Define a regular grid of points to interpolate to
    xi = np.linspace(x.min(), x.max(), 100)
    yi = np.linspace(y.min(), y.max(), 100)
    X, Y = np.meshgrid(xi, yi)

    # Interpolate the u, v and p values to the regular grid
    U = griddata((x, y), u, (X, Y), method='cubic')
    V = griddata((x, y), v, (X, Y), method='cubic')
    P = griddata((x, y), p, (X, Y), method='cubic')
    return x, y, u, v, p

In [ ]:
[x_FLUENT, y_FLUENT, u_FLUENT, v_FLUENT, p_FLUENT] = preprocess(dir='./FluentSolStep.txt')
field_FLUENT = [x_FLUENT, y_FLUENT, u_FLUENT, v_FLUENT, p_FLUENT]

x_PINN_1 = np.linspace(0, 1.0, 221)
y_PINN_1 = np.linspace(0, 0.21, 101)
x_PINN_2 = np.linspace(1.0, 1.5, 241)
y_PINN_2 = np.linspace(0.21, 0.41, 101)
x_PINN = np.concatenate((x_PINN_1, x_PINN_2))
y_PINN = np.concatenate((y_PINN_1, y_PINN_2))
x_PINN, y_PINN = np.meshgrid(x_PINN, y_PINN)
x_PINN = x_PINN.flatten()[:, None]
y_PINN = y_PINN.flatten()[:, None]

In [ ]:
psi, p_PINN, s11, s22, s12, tape, x, y, u_PINN, v_PINN = solver.get_output(tf.convert_to_tensor(x_PINN), tf.convert_to_tensor(y_PINN))
field_MIXED = [x_PINN, y_PINN, u_PINN, v_PINN, p_PINN]
postProcess(xmin=0, xmax=1.5, ymin=0, ymax=0.41, field_FLUENT=field_FLUENT, field_MIXED=field_MIXED, s=3, alpha=0.5, filename=f"uvp_step.png")

In [ ]:
plot_Wall_Shear_stress(solver)